In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from collections import defaultdict

In [3]:
usecols = ['ncodpers', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
       'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
       'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
       'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
       'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
       'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
       'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
       'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']
       
df_train = pd.read_csv('D:/Github/machine learning/Kaggle/Santander Product Recommendation/train_ver2.csv', usecols=usecols)
sample = pd.read_csv('D:/Github/machine learning/Kaggle/Santander Product Recommendation/sample_submission.csv')

In [4]:
df_train = df_train.drop_duplicates(['ncodpers'], keep='last')

df_train.fillna(0, inplace=True)

In [5]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
ids = df_train['ncodpers'].values

In [6]:
df_train.head()

,ncodpers,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
1922,1049700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
2989,1055504,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
5666,1045269,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
6196,1042585,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0
7440,1047273,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0


In [7]:
for c in df_train.columns:
    if c != 'ncodpers':
        print(c)
        y_train = df_train[c]
        x_train = df_train.drop([c, 'ncodpers'], 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[c] = clf
        model_preds[c] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train))
        

ind_ahor_fin_ult1
0.7921505585425102
ind_aval_fin_ult1
0.9794712736076367
ind_cco_fin_ult1
0.6501686786010519
ind_cder_fin_ult1
0.7889473557316506
ind_cno_fin_ult1
0.971472659406951
ind_ctju_fin_ult1
0.8655533428519266
ind_ctma_fin_ult1
0.6615937088529517
ind_ctop_fin_ult1
0.6127918284742274
ind_ctpp_fin_ult1
0.7223759191492828
ind_deco_fin_ult1
0.8888403119369325
ind_deme_fin_ult1
0.8675208337034979
ind_dela_fin_ult1
0.7819443159735965
ind_ecue_fin_ult1
0.7753431738629951
ind_fond_fin_ult1
0.8565750495454116
ind_hip_fin_ult1
0.9180935477344321
ind_plan_fin_ult1
0.848119443502516
ind_pres_fin_ult1
0.7097390540022359
ind_reca_fin_ult1
0.8344123432179367
ind_tjcr_fin_ult1
0.9051590483519067
ind_valo_fin_ult1
0.8408022240343198
ind_viv_fin_ult1
0.767959582486476
ind_nomina_ult1
0.9981800096266735
ind_nom_pens_ult1
0.998311131001897
ind_recibo_ult1
0.867506551654103


In [8]:
already_active = {}
for row in df_train.values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(df_train.columns[1:], row) if c[1] > 0]
    already_active[id] = active

In [9]:
train_preds = {}
for id, p in id_preds.items():
    # Here be dragons
    preds = [i[0] for i in sorted([i for i in zip(df_train.columns[1:], p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:7]]
    train_preds[id] = preds

In [10]:
test_preds = []
for row in sample.values:
    id = row[0]
    p = train_preds[id]
    test_preds.append(' '.join(p))

In [11]:
sample['added_products'] = test_preds
sample.to_csv('D:/Github/machine learning/Kaggle/Santander Product Recommendation/collab_sub.csv', index=False)